# Google Stock Price Prediction using RNN

In [ ]:
#RNN-GOOGLE STOCK PRICE
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data = pd.read_csv('Google_Stock_Price_Train.csv')
data.head()
data.info()
data_set = data[['Open']].values
train = data_set[:len(data_set)-50]
test = data_set[len(train):]
train.reshape(train.shape[0],1)
train.shape

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))
train_scaler = scaler.fit_transform(train)
plt.plot(train_scaler)
plt.show()
X_train = []
Y_train = []

In [ ]:
timesteps = 50
for i in range(timesteps, len(train_scaler)):
    X_train.append(train_scaler[i-timesteps:i,0])
    Y_train.append(train_scaler[i,0])
X_train, Y_train = np.array(X_train), np.array(Y_train)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1],1 ))

In [ ]:
from keras.models import Sequential
from keras.layers import SimpleRNN, Dropout, Dense
regressor = Sequential()
regressor.add(SimpleRNN(units=50, activation = 'tanh', return_sequences=True, input_shape=(X_train.shape[1],1)))
regressor.add(Dropout(0.2))
regressor.add(SimpleRNN(units=50, activation = 'tanh', return_sequences=True))
regressor.add(Dropout(0.2))
regressor.add(SimpleRNN(units=50, activation = 'tanh', return_sequences=True))
regressor.add(Dropout(0.2))
regressor.add(SimpleRNN(units = 50))
regressor.add(Dropout(0.2))
regressor.add(Dense(units = 1))

In [ ]:
regressor.compile(optimizer='adam', loss='mean_squared_error')
regressor.fit(X_train, Y_train, batch_size = 16, epochs = 20)
inputs = data_set[len(data_set) - len(test) - timesteps:]
inputs = scaler.transform(inputs)

In [ ]:
X_test = []
for i in range(timesteps, inputs.shape[0]):
    X_test.append(inputs[i - timesteps:i, 0])
X_test_rnn = np.array(X_test)
X_test_rnn = np.reshape(X_test_rnn, (X_test_rnn.shape[0], X_test_rnn.shape[1], 1))
predicted_stock_price = regressor.predict(X_test_rnn)
predicted_stock_price = scaler.inverse_transform(predicted_stock_price)

In [ ]:
from sklearn import metrics
mae = metrics.mean_absolute_error(test, predicted_stock_price)
print("MAE: ", mae)
mse = metrics.mean_squared_error(test, predicted_stock_price)
rmse = np.sqrt(mse)
print("RMSE: ",rmse)
mape = metrics. mean_absolute_percentage_error(test, predicted_stock_price)
accuracy = 100 - mape * 100
print(f"MAPE: {mape:.4f}")
print(f"Approximate Accuracy: {accuracy:.2f}%")


In [ ]:
# Plot predicted vs actual stock prices
plt.figure(figsize=(12, 6))
plt.plot(test, color='blue', label='Actual Google Stock Price')
plt.plot(predicted_stock_price, color='red', linestyle='--', label='Predicted Google Stock Price')
plt.title('Google Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Stock Price')
plt.legend()
plt.grid(True)
plt.show()